In [17]:
import networkx as nx
import random
import time
import math
import pandas as pd

In [18]:
# ==========================================
# 1. LÓGICA DE LOS ALGORITMOS (CORE)
# ==========================================

def count_leaves(T):
    return sum(1 for n in T.nodes() if T.degree(n) == 1)

def greedy_mlst_deterministic(G):
    degrees = dict(G.degree())
    if not degrees: return nx.Graph()
    root = max(degrees, key=degrees.get)
    tree_edges = []
    visited = {root}
    frontier = [] 
    for n in G.neighbors(root): frontier.append((root, n))
        
    while len(visited) < len(G.nodes()):
        if not frontier: break 
        frontier.sort(key=lambda x: G.degree(x[1]), reverse=True)
        u, v = frontier.pop(0) 
        if v not in visited:
            visited.add(v)
            tree_edges.append((u, v))
            for w in G.neighbors(v):
                if w not in visited: frontier.append((v, w))
    T = nx.Graph(); T.add_nodes_from(G.nodes()); T.add_edges_from(tree_edges)
    return T

def local_search_robust(G, initial_T, max_checks=200, max_improvements=50):

    current_T = initial_T.copy()
    current_leaves = count_leaves(current_T)
    improved = True
    improvements_count = 0
    
    while improved and improvements_count < max_improvements:
        improved = False
        
        # Candidatos: Aristas que conectan hojas con el resto
        leaves = [n for n in current_T.nodes() if current_T.degree(n) == 1]
        candidates = []
        for u in leaves:
            for v in G.neighbors(u):
                if not current_T.has_edge(u, v): candidates.append((u, v))
        
        if not candidates: break
        
        # Sampling para velocidad
        if len(candidates) > max_checks:
            candidates = random.sample(candidates, max_checks)
        else:
            random.shuffle(candidates)
            
        for u, v in candidates:
            try: path = nx.shortest_path(current_T, u, v)
            except: continue
            
            # Probar romper el ciclo
            # Solo probamos 5 aristas aleatorias del ciclo para no ser lentos
            path_edges = list(zip(path, path[1:]))
            if len(path_edges) > 5: edges_to_try = random.sample(path_edges, 5)
            else: edges_to_try = path_edges
            
            for x, y in edges_to_try:
                # 1. Aplicar Cambio
                current_T.add_edge(u, v)
                current_T.remove_edge(x, y)
                
                # 2. Verificar Ganancia REAL
                new_leaves_count = count_leaves(current_T)
                
                if new_leaves_count > current_leaves:
                    current_leaves = new_leaves_count
                    improved = True
                    improvements_count += 1
                    break # Aceptar y reiniciar búsqueda
                else:
                    # 3. Revertir si no mejora
                    current_T.add_edge(x, y)
                    current_T.remove_edge(u, v)
            
            if improved: break
            
    return current_T

def perturbation(G, T):
    T_new = T.copy()
    non_tree = list(set(G.edges()) - set(T_new.edges()))
    if not non_tree: return T_new
    
    for _ in range(5): # Intentar 5 veces encontrar un swap válido
        u, v = random.choice(non_tree)
        try:
            path = nx.shortest_path(T_new, u, v)
            if len(path) > 2:
                idx = random.randint(0, len(path)-2)
                x, y = path[idx], path[idx+1]
                T_new.add_edge(u, v); T_new.remove_edge(x, y)
                return T_new
        except: pass
    return T_new

def super_ils(G, max_time=2.0):
    start = time.time()
    # Arranque fuerte con Greedy Determinista
    best_T = greedy_mlst_deterministic(G)
    # Primera optimización segura
    best_T = local_search_robust(G, best_T)
    best_l = count_leaves(best_T)
    curr_T = best_T.copy()
    
    # Bucle de tiempo
    while time.time() - start < max_time:
        # Perturbar
        cand_T = perturbation(G, curr_T)
        # Optimizar
        cand_T = local_search_robust(G, cand_T, max_checks=100, max_improvements=10)
        
        cand_l = count_leaves(cand_T)
        if cand_l > best_l:
            best_T = cand_T.copy(); best_l = cand_l; curr_T = cand_T.copy()
        elif cand_l >= count_leaves(curr_T):
             curr_T = cand_T.copy()
             
    return best_T


In [19]:
# ==========================================
# 2. GENERACIÓN Y EJECUCIÓN
# ==========================================

def generar_datos_y_guardar():
    print("Generando experimentos con grafos complejos...")
    data = []
    
    configs = [
        # 1. Malla (Tu caso de éxito)
        ("Malla 8x8", nx.grid_2d_graph(8, 8)),
         # 1.2. Malla (Tu caso de éxito)
        ("Malla 16x16", nx.grid_2d_graph(16, 16)),
        
        # 2. Watts-Strogatz (Mundo Pequeño - Muchos ciclos locales)
        # n=100, k=6 vecinos, p=0.1 probabilidad de re-cableado
        ("Watts-Strogatz (100)", nx.watts_strogatz_graph(100, 6, 0.1)),
        
        # 3. Barabasi-Albert (Redes Reales - Hubs fuertes)
        # n=100, m=3 aristas nuevas por nodo
        ("Barabasi-Albert (100)", nx.barabasi_albert_graph(100, 3)),
        
        # 4. Grafo Regular (Muy difícil para Greedy por simetría)
        # n=100, grado=4
        ("Regular d=4 (100)", nx.random_regular_graph(4, 100)),
        
        # 5. Aleatorio Erdos-Renyi (Control)
        ("Aleatorio ER (100)", nx.erdos_renyi_graph(100, 0.08))
    ]
    
    for name, G in configs:
        # Normalizar etiquetas a enteros
        G = nx.convert_node_labels_to_integers(G)
        
        # Asegurar conexidad
        if not nx.is_connected(G):
            # Si no es conexo, tomamos la componente gigante
            largest_cc = max(nx.connected_components(G), key=len)
            G = G.subgraph(largest_cc).copy()
            name += " (CC)" # Marcamos que fue recortado
            
        print(f"Procesando {name}...")

        # 1. Greedy
        start = time.time()
        T_g = greedy_mlst_deterministic(G)
        t_g = time.time() - start
        res_g = count_leaves(T_g)
        
        # 2. BL Simple
        start = time.time()
        T_bl = local_search_robust(G, T_g, max_checks=2000)
        t_bl = time.time() - start + t_g
        res_bl = count_leaves(T_bl)
        
        # 3. ILS (Tiempo fijo 2s)
        start = time.time()
        T_ils = super_ils(G, max_time=2.0)
        t_ils = time.time() - start 
        res_ils = count_leaves(T_ils)
        
        # Guardar fila
        data.append({
            "Caso": name,
            "Nodos": len(G.nodes()),
            "Aristas": len(G.edges()),
            "Greedy_Hojas": res_g, "Greedy_Tiempo": t_g,
            "BL_Hojas": res_bl, "BL_Tiempo": t_bl,
            "ILS_Hojas": res_ils, "ILS_Tiempo": t_ils
        })
        
    df = pd.DataFrame(data)
    df.to_csv("resultados_mlst100.csv", index=False)
    print("\n¡Listo! Datos guardados en 'resultados_mlst100.csv'")
    print(df[["Caso", "Greedy_Hojas", "BL_Hojas", "ILS_Hojas"]])



In [20]:
# Ejecutar generador
generar_datos_y_guardar()

Generando experimentos con grafos complejos...
Procesando Malla 8x8...
Procesando Malla 16x16...
Procesando Watts-Strogatz (100)...
Procesando Barabasi-Albert (100)...
Procesando Regular d=4 (100)...
Procesando Aleatorio ER (100)...

¡Listo! Datos guardados en 'resultados_mlst100.csv'
                    Caso  Greedy_Hojas  BL_Hojas  ILS_Hojas
0              Malla 8x8            24        24         36
1            Malla 16x16            56        56        125
2   Watts-Strogatz (100)            66        67         69
3  Barabasi-Albert (100)            83        83         83
4      Regular d=4 (100)            44        49         57
5     Aleatorio ER (100)            74        74         75
